In [69]:
import os
from sqlalchemy import create_engine

# -------------------------------------------------
# PostgreSQL connection from .env
# -------------------------------------------------
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT", "5432")
PG_DB = os.getenv("PG_DB")
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")

# Build Postgres URL
pg_url = (
    f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}"
    f"@{PG_HOST}:{PG_PORT}/{PG_DB}"
)

engine = create_engine(pg_url, pool_pre_ping=True)



In [70]:
import os
import re
from dataclasses import dataclass, field
from typing import Optional, Dict, Any, List

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

from langchain_groq import ChatGroq
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_classic.chains.sql_database.query import create_sql_query_chain

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

True

In [71]:
# ----------------------------
# 2) PostgreSQL connection
# ----------------------------
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT", "5432")
PG_DB = os.getenv("PG_DB")
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_SCHEMA = os.getenv("PG_SCHEMA", "public")

if not all([PG_HOST, PG_DB, PG_USER, PG_PASSWORD]):
    raise ValueError("Missing .env vars. Need PG_HOST, PG_DB, PG_USER, PG_PASSWORD (and optional PG_PORT, PG_SCHEMA).")

pg_url = f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}"
engine = create_engine(pg_url, pool_pre_ping=True)

db = SQLDatabase(engine, schema=PG_SCHEMA)
print("Available tables:", db.get_usable_table_names())

Available tables: ['delivery_data']


In [72]:
# 3) Groq LLM
# ----------------------------
llm = ChatGroq(
    model=os.getenv("GROQ_MODEL", "meta-llama/llama-4-scout-17b-16e-instruct"),
    temperature=float(os.getenv("LLM_TEMPERATURE", "0.0")),
    api_key=os.getenv("GROQ_API_KEY"),
)

parser = StrOutputParser()

In [73]:
# 4) Safety utilities
# ----------------------------
DEFAULT_LIMIT = int(os.getenv("DEFAULT_LIMIT", "200"))

# Recommended: restrict SQL to your main fact table(s).
# Put your actual table name(s) here. Example: delivery_data
ALLOWED_TABLES = [t.strip() for t in os.getenv("ALLOWED_TABLES", "delivery_data").split(",") if t.strip()]

FORBIDDEN_SQL = [
    r"\bINSERT\b", r"\bUPDATE\b", r"\bDELETE\b", r"\bDROP\b", r"\bALTER\b",
    r"\bTRUNCATE\b", r"\bCREATE\b", r"\bGRANT\b", r"\bREVOKE\b",
]

def extract_sql(text_out: str) -> str:
    """Extract just SQL from LLM output (code fences / SQLQuery: label)."""
    if not text_out:
        return ""
    match = re.search(r"```sql\s*(.*?)```", text_out, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1).strip()
    if "SQLQuery:" in text_out:
        return text_out.split("SQLQuery:", 1)[1].strip()
    return text_out.strip().strip("`").strip()

def is_select_only(sql: str) -> bool:
    s = (sql or "").strip()
    if not s.lower().startswith("select"):
        return False
    for pat in FORBIDDEN_SQL:
        if re.search(pat, s, flags=re.IGNORECASE):
            return False
    # block multi-statement
    if ";" in s.rstrip().rstrip(";"):
        return False
    return True

def ensure_limit(sql: str, default_limit: int = 200) -> str:
    if re.search(r"\blimit\b", sql, flags=re.IGNORECASE):
        return sql
    return sql.rstrip() + f"\nLIMIT {default_limit}"

def enforce_table_allowlist(sql: str, allowed_tables: List[str]) -> None:
    """Simple allowlist check. Production-hardening can use a SQL parser later."""
    if not allowed_tables:
        return
    s = sql.lower()
    if not any(t.lower() in s for t in allowed_tables):
        raise ValueError(f"Query must reference allowed tables only: {allowed_tables}")

def log_sql(sql: str) -> str:
    print("\n--- SQL ---\n", sql)
    return sql

def looks_like_why(text: str) -> bool:
    t = (text or "").strip().lower()
    return bool(re.search(r"\b(why|reason|explain|root cause|what happened|recommend|suggest|how to|action)\b", t))

def looks_like_followup(text: str) -> bool:
    t = (text or "").strip().lower()
    # heuristic follow-up patterns (extend as needed)
    return bool(re.search(r"\b(now|same|also|only|filter|breakdown|by\b|for\b|last\b|previous|compare|show)\b", t))

In [74]:
#  5) Prompts (Production-friendly)
# ----------------------------
contextualize_prompt = PromptTemplate.from_template("""
You rewrite a follow-up message into a complete standalone analytics question
for a PostgreSQL delivery database.

Previous question:
{last_question}

User message:
{user_message}

Rules:
- If the user message is already a complete question, return it unchanged.
  merge it with the previous question.
- Keep it concise and unambiguous.
Return ONLY the rewritten standalone question.
""")

descriptive_prompt = PromptTemplate.from_template("""
You are the DESCRIPTIVE agent.
Answer using ONLY the SQL Result. Do not invent any numbers or facts.

User Question: {question}
SQL Query: {query}
SQL Result: {result}

Write a clear, concise business answer.
If result is empty, say no matching data found and suggest 1-2 filters (date range, customer, SBU, zone).
""")

prescriptive_prompt = PromptTemplate.from_template("""
You are the PRESCRIPTIVE agent (deep why + what next).
You MUST ground your reasoning in the SQL Result and the descriptive answer.
If evidence is insufficient, say so and give hypotheses with uncertainty.

Original Question: {question}
SQL Query: {query}
SQL Result: {result}
Descriptive Answer: {descriptive_answer}

Now write:
1) Drivers (why this could be happening)
2) Evidence (what supports each driver from result)
3) Actions (what to do next)
4) Questions / next queries to confirm
""")

contextualize_chain = contextualize_prompt | llm | parser
desc_chain = descriptive_prompt | llm | parser
presc_chain = prescriptive_prompt | llm | parser


In [ ]:
#  6) SQL generation + execution (general, any DB question)
# ----------------------------
write_query = create_sprint("2",session.ask("why"))ql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

def run_sql_from_question(question: str) -> Dict[str, Any]:
    """
    Production-style SQL run:
    - generate SQL
    - extract SQL
    - validate SELECT-only + allowlist + LIMIT
    - execute and return result
    """
    raw = write_query.invoke({"question": question})
    sql = extract_sql(raw)
    sql = log_sql(sql)

    if not is_select_only(sql):
        raise ValueError(f"Unsafe SQL blocked:\n{sql}")

    enforce_table_allowlist(sql, ALLOWED_TABLES)
    sql = ensure_limit(sql, DEFAULT_LIMIT)

    result = execute_query.invoke(sql)
    return {"question": question, "query": sql, "result": result}

In [76]:
# 7) Session (continuous chat with context)
# ----------------------------
@dataclass
class SalesGPTSession:
    # last context for "why?" and follow-ups
    last_question: Optional[str] = None
    last_sql: Optional[str] = None
    last_result: Optional[str] = None
    last_descriptive: Optional[str] = None

    # optional history
    history: List[Dict[str, str]] = field(default_factory=list)

    def ask(self, user_message: str) -> str:
        self.history.append({"role": "user", "content": user_message})

        # A) Prescriptive "why" flow
        if looks_like_why(user_message):
            if not self.last_question:
                msg = "Ask a data question first (so I have context). Then ask 'why?' and I’ll explain based on the previous result."
                self.history.append({"role": "assistant", "content": msg})
                return msg

            answer = presc_chain.invoke({
                "question": self.last_question,
                "query": self.last_sql,
                "result": self.last_result,
                "descriptive_answer": self.last_descriptive
            })
            self.history.append({"role": "assistant", "content": answer})
            return answer

        # B) Descriptive flow (any DB question)
        question = user_message

        # If it looks like a follow-up, rewrite it using the previous question
        if self.last_question and looks_like_followup(user_message):
            question = contextualize_chain.invoke({
                "last_question": self.last_question,
                "user_message": user_message
            }).strip()

        out = run_sql_from_question(question)
        desc = desc_chain.invoke(out)

        # Store for next turns
        self.last_question = out["question"]
        self.last_sql = out["query"]
        self.last_result = out["result"]
        self.last_descriptive = desc

        self.history.append({"role": "assistant", "content": desc})
        return desc

# ----------------------------
# 8) Usage
# ----------------------------
# Create a session and chat continuously:
session = SalesGPTSession()


In [78]:

# Example:
print("1",session.ask("Top customer"))



--- SQL ---
 SELECT 
    "Customer_Name", 
    SUM("Delivery_value") AS TotalDeliveryValue
FROM 
    public.delivery_data
GROUP BY 
    "Customer_Name"
ORDER BY 
    TotalDeliveryValue DESC
LIMIT 5;
1 Our top customer is M/S Jahan Trading - Mirpur with a total delivery value of 318,590,282.5, followed closely by M/S Shahin Traders -2 - Cumilla and M/S Bhuiyan Trader's - Cumilla. The top 5 customers account for a significant portion of our delivery value. 

Here are the top 5 customers:
1. M/S Jahan Trading - Mirpur - 318,590,282.5
2. M/S Shahin Traders -2 - Cumilla - 305,042,780.0
3. M/S Bhuiyan Trader's - Cumilla - 255,213,800.0
4. Spectra Engineers Ltd - 156,368,890.0
5. National Development Engineers Ltd - 153,744,980.0


In [79]:
print("2",session.ask("why"))

2 **Drivers (Why this could be happening)**

Based on the SQL result and descriptive answer, here are some potential drivers for the top customers:

1. **Strong Business Relationship**: M/S Jahan Trading - Mirpur, M/S Shahin Traders -2 - Cumilla, and M/S Bhuiyan Trader's - Cumilla might have a strong business relationship with our company, leading to a high volume of deliveries.
2. **Large Order Quantity**: These top customers might be placing large orders, resulting in a significant total delivery value.
3. **Frequent Deliveries**: The top customers might require frequent deliveries, contributing to their high total delivery value.
4. **Strategic Location**: The top customers might be located in strategic areas, making it easier and more cost-effective for our company to deliver to them.

**Evidence (What supports each driver from result)**

From the SQL result, we can see that:

1. **Consistent High Delivery Values**: The top 3 customers (M/S Jahan Trading - Mirpur, M/S Shahin Trader

In [82]:
print("3",session.ask("what to do next?"))


--- SQL ---
 SELECT 
    "Delivery_Code",
    "Customer_Name",
    "Customer_Address",
    "Delivery_Date",
    "Customer_Receive_Status"
FROM 
    public.delivery_data
WHERE 
    "Customer_Receive_Status" = 'Pending'
LIMIT 5;
3 **Pending Deliveries**

There are 5 pending deliveries as of now. 

**Action Items:**

1. **Verify Delivery Status**: Review and confirm the delivery status of the following customers: 
   - M/S Hashem khan Traders
   - M/S Maa Babar Doa Store- Gazaria 
   - M/S Shahin Traders -2 - Cumilla ( duplicate entry)
   - Maa Enterprise- Turag

2. **Track and Dispatch**: Track the delivery of these orders and dispatch them to the customers.

**Pending Delivery Details:**

| Delivery Code | Customer Name | Customer Address | Delivery Date | Customer Receive Status |
| --- | --- | --- | --- | --- |
| SD122512931 | M/S Hashem khan Traders | Rupa Khan Super Market, Shorifpur,National University, Gazipur  | 12/22/2025 | Pending |
| SD122512948 | M/S Maa Babar Doa Store- Gaz